# Co-expression of sibling genes

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'CoEx_Histos'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import psutil
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Make some choices...
#dataset_list = ['Y21ss'] #,'Y21tenX','Y23', 'H19'
dataset = 'Y21ss'
#dataset = 'H19'

if dataset == 'Y21ss' or dataset == 'H19': # SMART-Seq
    min_tally = 10
    threshold = 10
else: # TenX
    min_tally = 100
    threshold = 1

if dataset[0] == 'Y': # Path for tables specific to species but not to dataset
    species = 'mouse'
    tables_path = 'Resources/Mouse_tables/'
else:
    species = 'human'
    tables_path = 'Resources/Human_tables/'    
    
#min_tally = 10 if dataset == 'Y21ss' else 100
#threshold = 10 if dataset == 'Y21ss' else 1
sns.set_theme(font_scale=1.3, rc={'patch.linewidth': 0})

# Assemble the general filepaths
name_root = notebook_name
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

#for dataset in dataset_list:

# Read annotated expression data
#source_path = 'Resources/'+dataset+'_NP_counts.csv.zip' # 'NP' or 'nMod'
#support_path = 'Resources/'+dataset+'_NP_support.json' # 'NP' or 'nMod'
source_path = 'Resources/'+dataset+'_nMod_counts.csv.zip' # 'NP' or 'nMod'
support_path = 'Resources/'+dataset+'_nMod_support.json' # 'NP' or 'nMod'

print('Reading',source_path)
df = pd.read_csv(source_path)
anno_counts = df.copy()
# Map expression in selected cell types only
#anno_counts = df[df['type'] == '01 Neuronal'].copy()
#anno_counts = df[df['type'] == '02 Glial'].copy()

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_zones = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_zones = {key: subclass_zones[key] for key in sorted(subclass_zones)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}
cluster_zones = dict(zip(anno_counts['cluster'],[s.split(' ',1)[1] for s in anno_counts['cluster']]))
cluster_zones = {key: cluster_zones[key] for key in sorted(cluster_zones)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')
region_shorts = df.set_index('region_label')['region_short'].to_dict()
region_zones = df.set_index('region_label')['zone'].to_dict()

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())

# Read CSV table to form gene label dictionary
df = pd.read_csv(tables_path + 'All_nMods_labeled.csv')
gene_labels = df.set_index('Symbol').to_dict()['Label']

# Configure multiindex
major_annos = ['type','region','class','subclass','cluster']
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos)
anno_counts.set_index(major_annos, inplace=True)

# Sanity check
if species == 'mouse':
    print('Sanity check:\n'+str(anno_counts[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1']].sum()))
    
else:
    print('Sanity check:\n'+str(anno_counts[['CCK','NPY','VIP','CCKBR','KNG1','BDKRB1']].sum()))

print('\n',anno_counts.shape)
anno_counts.head(1)


In [ ]:
# Co-expression by cells

# Make some choices
threshold = 1
bins = 20
regions = ['000 All'] #+ regions

# Slice it up
#genes = 'NPP'
genes = 'NP-GPCR'
if genes == 'NPP': geneset = '30NPPs'
if genes == 'NP-GPCR': geneset = '52_NP-GPCRs_Conj-Top30_NPPs'
genelist = list(pd.read_csv(tables_path+geneset+'.csv')['Symbol'])   
df = anno_counts[genelist].copy()

type_id = '01 Neuronal'
class_ids = ['00 All','01 Glutamatergic','02 GABAergic']

# Filter based on type IDs
df = df.loc[type_id]
    
# Generate filename root
root = dataset
if type_id == 'All': root += '_all_'
elif type_id == '01 Neuronal': root += '_neur_'
else: root += '_non-neur_'
root += genes+'_T' + str(threshold)

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root+'_'+sn

pdf_file = pdf_path+handle+'.pdf'
pdf = PdfPages(pdf_file)

for region in regions:
    if region == '000 All': cell_hot = (df >= threshold).astype(int).droplevel(0)
    else: cell_hot = (df.loc[region] >= threshold).astype(int)
    for class_id in class_ids:
        if class_id == '00 All': nps = cell_hot.T
        else: nps = cell_hot.loc[class_id].T
        any_np = nps.sum()
        histo,bins,artists = plt.hist(any_np, bins=bins, range=(0,bins), rwidth=0.9)
        plt.xlabel('# '+genes+' Genes Co-Expressed\n'+geneset+'\nThreshold: '+str(threshold)+
                   '\n' + class_id.split()[1]+'\n')
        plt.ylabel('Frequency')
        #plt.title(geneset+'\nThreshold: '+str(threshold)+'\n' + class_id.split()[1]+'\n' + 
        plt.title(str(100 - round(100*histo[0]/len(any_np),2))+'% Non-Zero, '+
                  'Modal # = '+str(histo.argmax()),fontsize=22)
        # Save and show the plot
        pdf.savefig(dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.2)
        plt.show()

pdf.close(); print('Graphics to PDF:', pdf_file)
